# Neste notebook faremos a simulação por dinâmica molecular do gás argônio superfluido à 100 K e 1000 atm.

Antes de dar início à simulação, deveremos instalar o pacote M3L responsável pela dinâmica molecular. Descomente a linha abaixo caso ele ainda não foi instalado no seu ambiente virtual.

In [1]:
#pip install -i https://test.pypi.org/simple/ m3l --extra-index-url https://pypi.org/simple/

## Etapa 1
O primeiro passo consiste em definirmos o sistema físico contendo as coordenadas atômicas. Descomente as cinco primeiras linhas abaixo para fazer o download do arquivo "Ar_100K_1atm.json".

In [2]:
# download do arquivo 'Ar_100K_1000atm.json'
#url = 'https://raw.githubusercontent.com/flavianowilliams/M3L/development/src/notebooks/argon/Ar_94K_1atm.json'
#from urllib.request import urlopen
#
#file = urlopen(url)
#
#with open('Ar_94K_1atm.json','wb') as output:
#  output.write(file.read())
#
# carregando sistema físico
from m3l.structure import System as sys
import json
#
system = sys()
system.loadSystem('Ar_94K_1atm.json')
#
# convertendo as unidades de medida
system.convertUnits()

## Etapa 2
O segundo passo consiste em definirmos o modelo para o campo de força. Neste caso consideraremos uma molécula contendo apenas um sítio atômico com interações intermoleculares do tipo Van der Waals representado pela função Lennard-Jones de dois parâmetros, como mostra a figura abaixo.
![lennard-jones](../contrib/lennard-jones.jpg)

In [3]:
# definindo o modelo de interação entre os átomos (campo de força)      
import m3l.force_field as ff
from m3l.force_field import Intermolecular as interff
class Forces(ff.ForceField):
    def __init__(self):
        super().__init__()
        self.structure(
            self.molecule(
                interff.site(1, 0.0, 0.2385, 3.4), #site ID, atomic charge, Lennard-Jones parameters
            )
        )
#        
model = Forces()
model()

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 3.80200013e-04, 6.42506883e+00]])

## Etapa 3
O terceiro passo consiste em definir o modelo estatístico (ensemble canônico) que descreverá o sistema no equilíbrio termodinâmico.

In [4]:
# definindo o modelo estatístico (ensemble)
from m3l.molecular_dynamics import Ensemble as en
#
force_field = model()
ensemble = en(temp_bath = 94.0, press_bath = 1.0, timestep = 1.0e-3, force_field = force_field)

## Etapa 4
O quarto passo consiste na dinâmica molecular, onde as coordenadas atômicas são recalculadas a cada passo da simulação em intervalos de tempo denominado de timestep.

In [5]:
# Etapa 4
# executando ciclo MD
import time
#
# executando looping
n_steps = 1000
i_step = 0
start = time.time()
for step in range(n_steps):
    system = ensemble(system)
end = time.time()
print(f'Elapsed time: {round(end - start, 0)} seconds')

Elapsed time: 2.0 seconds


## Etapa 5
Esta etapa é opcional, onde o autor poderá salvar em um arquivo JSON o sistema físico obtido no final da simulação. Como opção, o autor poderá converter as unidades de medida em valores mais convenientes para a leitura e melhor interpretação do ponto de vista didático.

In [6]:
# Etapa opcional
# convertendo para as unidades de medida de entrada
system.convertUnitsInv()
#
# salvando sistema em arquivo JSON
system.save('Ar_94K_1atm.json')

Algumas propriedades termodinâmicas do sistema físico poderão ser lidas através dos atributos do objeto system.

In [7]:
# Etapa opcional
# Obtendo variáveis termodinâmicas do sistema
print(f'Energia potencial: {system.epotential} kcal/mol')
print(f'Energia cinética: {system.ekinetic} kcal/mol')
print(f'Temperatura: {system.temperature} K')
print(f'Pressão: {system.pressure} atm')

Energia potencial: -457.1698936384324 kcal/mol
Energia cinética: 71.23023404950321 kcal/mol
Temperatura: 96.00089615112887 K
Pressão: -44.15018214663735 atm
